# Introduction

Redis Enterprise is an enterprise-grade Redis, available both on-premises and in the cloud (on AWS, Google Cloud, or Azure).
Redis Enterprise simplifies operations, scaling, and multi-tenancy includes many integrations (for example, Kubernetes), and provides multiple tiers of support.
<br>Redis Enterprise offers robust vector database features, with an efficient API for vector index creation, management, distance metric selection, similarity search, and hybrid filtering. When coupled with its versatile data structures - including lists, hashes, JSON, and sets - Redis Enterprise shines as the optimal solution for crafting high-quality Large Language Model (LLM)-based applications. It embodies a streamlined architecture and exceptional performance, making it an instrumental tool for production environments.

### Important use cases include:
* __Chatbots with RAG__
    <br>Ground chatbots in your data using Retrieval Augmented Generation (RAG) to enhance the quality of LLM responses.

* __Semantic caching__
    <br>Identify and retrieve cached LLM outputs to reduce response times and the number of requests to your LLM provider, which saves time and money.

* __Recommendation systems__
    <br>Power recommendation engines with fresh, relevant suggestions at low-latency, and point your users to the products they’re most likely to buy.

* __Document Search__
    <br>Make it easier to discover and retrieve information across documents and knowledge bases, using natural language and semantic search.

# Google's Vertex AI
Google's Vertex AI has expanded its capabilities by introducing Generative AI. This advanced technology comes with a specialized in-console studio experience, a dedicated API and Python SDK designed for deploying and managing instances of Google's powerful Gemini language models.

# Lab overview & Objective

In this Lab, we will implement a production-ready proof of concept by building an VSS application deployed on Google Cloud's infrastructure using Redis as a backbone.
Here we will use a sample IMDB movies dataset, load this in Redis and finally invoke different types of search queries to get insight from this dataset.
We will use the following libraries and frameworks:
* Google Colab for hosting Jupyter Notebook
* Redis Enterprise Cloud as Vector DB provider
* redis-py Python library for Redis
* redis-vl Python library for Vector specific tasks
* Langchain for other vector management tasks



### Install dependencies


In [ ]:
!pwd
!pip install --upgrade pip
# Install required libraries
!python3 -m pip -q install redis pandas
!pip install -U git+https://github.com/RedisVentures/redisvl.git langchain


###Configure Redis


#### **Recommended**: Configure Redis Enterprise

Here we will leverage Redis Enterprise Cloud available through the GCP marketplace.

These are the high level steps for subscribing to Redis Enterprise Cloud database using GCP Marketplace.

**Note: For this workshop, each participants will be provided a unique Redis URL. Reach out to the Redis team to get one.**

* Log in to GCP Console, navigate to Marketplace and search for Redis Enterprise
* Click on the option that displays Redis Enterprise Cloud and subscribe to this
* The console will navigate to the Redis Enterprise Cloud URL
* Sign up for Redis Enterprise Cloud. The system will ask for confirmation
* Finally, create the database with your preferred region. For this exercise, an 'Essential' subscription will be sufficient. Select the Redis Stack
* Once the DB is active, note down the URL. This will be needed in the subsequent steps


In [ ]:
## Update the 'host' field with the correct Redis host URL
host = ''
port =
password = 'admin'
requirePass = True


#### *Alternative: Redis Stack (Community Edition)*

In case Redis Enterprise Cloud URL is not available, you need to install Redis Stack (Community edition) using following steps. Uncomment the following lines of code and execute:

In [ ]:
## Uncomment & execute the following code in case Redis Enterprise is not available
##################################################################################

# %%sh
# curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
# echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
# sudo apt-get update  > /dev/null 2>&1
# sudo apt-get install redis-stack-server  > /dev/null 2>&1
# redis-stack-server --daemonize yes


In [ ]:
## For Redis Stack server, uncomment the following code:
# host = 'localhost'
# port = 6379
# requirePass = False

#### Create Redis connection object


In [ ]:
import redis

if requirePass:
    client = redis.Redis(host = host, port=port, decode_responses=True, password=password)
    url = f'redis://:{password}@{host}:{port}'
else:
    client = redis.Redis(host = 'localhost', decode_responses=True)
    url = f'redis://{host}:{port}'

print(client.ping())
# Clear Redis database (optional)
client.flushdb()



### Text Vector Embedding

Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:

* Semantic search: Search text ranked by semantic similarity.
* Recommendation: Return items with text attributes similar to the given text.
* Classification: Return the class of items whose text attributes are similar to the given text.
* Clustering: Cluster items whose text attributes are similar to the given text.
* Outlier Detection: Return items where text attributes are least related to the given text.


#### **Recommended: Google Vertex AI Text Embedding model**

Authenticate with GCP & set project id and region
For this session, it is recommneded to use API of Vertex AI Embedding model to create embeddings for our dataset.

Before doing that we must authenticate with GCP and get the suitable Google Project Id and Region

In [ ]:
## Authenticate with GCP & set project id and region
from google.colab import auth
from getpass import getpass

auth.authenticate_user()
print('Authenticated')

# input your GCP project ID and region for Vertex AI
PROJECT_ID = getpass("PROJECT_ID:")
REGION = 'us-central1' #input("REGION:")

print(f'PROJECT_ID: {PROJECT_ID} & REGION: {REGION}')


In [ ]:
#@title We will use redis-vl library to interact with the embedding model
## Select embedding model provider as "textembedding-gecko@003"

from redisvl.utils.vectorize import VertexAITextVectorizer

vectorizer = VertexAITextVectorizer(
    model = "textembedding-gecko@003",
    api_config = {"project_id": PROJECT_ID, "location": REGION}
)

#### *Alternative: Sentence-Transformer Text Embedding model*

In case GCP's Vertex AI Text embedding model is not available, you may also use Sentence-Transformer model.


In [ ]:
#@title Install sentence-transformers library

# Uncomment the following line of code in order to use sentence-transformers model
#!pip install sentence-transformers

In [ ]:
#@title We will use redis-vl library for interacting with embedding model
## Select embedding model provider as "sentence-transformers/all-mpnet-base-v2"

# Uncomment the following lines of code in order to use sentence-transformers model

# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# from redisvl.utils.vectorize import HFTextVectorizer

# vectorizer = HFTextVectorizer(model="sentence-transformers/all-mpnet-base-v2")


### Download the sample dataset - IMDB dataset
We will be using a sample IMDB Movies dataset available from Kaggle.
Next, we will load it into Pandas Dataframe and investigate the column and its data-type.

In [ ]:
!wget https://storage.googleapis.com/abhi-data-2024/MOVIES.csv -O movies.csv


In [ ]:
import pandas as pd

df = pd.read_csv('movies.csv')


In [ ]:
df.describe()

In [ ]:
# Truncate the 'overview' field for the long descriptions
df['overview_len'] = df['overview'].str.len()
df['overview'] = df['overview'].apply(lambda x: x[:300])
df.head(5)

### Create Embeddings
We will now create vector embeddings of "overview" field present in Movies dataset and modify the existing python dataframe

In [ ]:
## 1. Create embeddings for the 'overview' column in movies' Dataframe
## 2. Store these embeddings in a new column 'overview_embedding'
## 3. Finally, append this new embeddings column in the existing dataframe

#embeddings = vectorizer.embed_many([element for element in df['overview']]) ## This may encounter quota issue
embeddings = vectorizer.embed_many(df['overview'].tolist(), batch_size=200)
df.insert(len(df.columns)-1, "overview_embedding", embeddings)
df.head()

### Store the dataframe in Redis database
Once done, we will move to the next part of our exercise which is creating the suitable indexes.
We will use these indexes to build:
* VSS queries
* Standard search query
* Hybrid queries


In [ ]:
## Store the Dataframe in Redis
import json

pipeline = client.pipeline()

for index, row in df.iterrows():
    redis_key = f"doc:{index}"
    pipeline.json().set(redis_key, '$', row.to_dict())

pipeline.execute()

In [ ]:
## Verify of the record is presentin Redis
print(client.json().get('doc:4', '$.overview_embedding'))

### Create Indexes in Redis
Now is the time to query Redis database. For that we will create few indexes using __redis-vl__ Python library.
<br>Redis offers an enhanced Redis experience via the following search and query features:

* A rich query language
* Incremental indexing on JSON and hash documents
* Vector search
* Full-text search
* Geospatial queries
* Aggregations
* You can find a complete list of features here: https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/

The search and query features of Redis Stack allow you to use Redis as a:

* Document database
* Vector database
* Secondary index
* Search engine


In [ ]:
## create index using redis-vl Python library.

from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex


index_name = "idx_movie"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": "doc:",
    "storage_type": "json"
  },
  "fields": [
    {
        "name": "budget",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "original_title",
        "type": "text"
    },
    {
        "name": "overview",
        "type": "text"
    },
    {
        "name": "revenue",
        "type": "numeric"
    },
    {
        "name": "vote_count",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "popularity",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "overview_embedding",
        "type": "vector",
        "attrs": {
            "dims": vectorizer.dims,
            "distance_metric": "cosine",
            "algorithm": "flat",
            "datatype": "float32"
        }
    }
  ]
})

In [ ]:
print(vectorizer.dims)

In [ ]:
# Create an index from schema and the client
index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [ ]:
#@title redis-rvl library also provides CLI support as well. You can get the information of created indexes using following commands

!rvl index listall -u $url

!# inspect the index fields
!rvl index info -i idx_movie -u $url


### Querying Redis
Now is the time to query Redis database. Again we will use __redis-vl__ Python library to achieve this.
<br>We will invoke following types of queries against our records present in Redis:

* VSS queries
* Standard search query
* Hybrid queries


#### Pure VSS(KNN) and Hybrid Query
Let's search for something in Redis that has semantically similar meaning to what is being asked. We will first vectorise the input question/query and search within `overview_embeddings` column to fetch the records which has potentially similar meaning.
<br> Pure KNN queries scans through the entire search space which might take relatively longer time to execute. Depending on our use case, we can restrict this search space by providing filter conditions. Following examples discuss all these scenarios.

In [ ]:
#@title [Example 1] Pure KNN & Hybrid queries

from redisvl.query import VectorQuery
from redisvl.query.filter import Num

## Input queries which would be used for our example searches
queries = [
    "movies showing struggle",
    "woman with a secret on a dangerous mission",
    "man wrongly charged with murder",
    "a boy falls in love with a girl",
    "showing a magical, enchanted or fantasy world",
    "a sci fi thriller or adventure movie",
    "movie with a murder mystery",
    "based on shakespeare play",
    "war based movies"
]

def getVectorQuery(query):
  vote_count_filter = Num("vote_count") > 1

  query_embedding = vectorizer.embed(query)

  vector_query = VectorQuery(
      vector=query_embedding,
      vector_field_name="overview_embedding",
      num_results=5,
      return_fields=["original_title", "overview", "popularity", "revenue", "vote_count"],
      return_score=True,
      filter_expression=vote_count_filter
  )
  # show the raw redis query
  str(vector_query)
  return vector_query


In [ ]:
# execute the query with RedisVL
print(json.dumps(index.query(getVectorQuery(queries[6])), indent=2))

#### Standard Query
Now we have seen how VSS query works. Let's also try some standard redis Search queries. For that, we will create a new index and invoke few queries against the newly created index.

In [ ]:
from redis.commands.search.field import TagField, TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

INDEX_NAME = 'idx_movie_2'
DOC_PREFIX = 'doc:'

try:
    # check to see if index exists
    client.ft(INDEX_NAME).info()
    print('Index already exists!')
except:
    # schema
    schema = (
        TextField('$.original_title', as_name='Title'),
        NumericField('$.budget', as_name='budget'),
        TextField('$.release_date', as_name='release_date'),
        NumericField('$.revenue', as_name='revenue'),
        NumericField('$.vote_count', as_name='vote_count'),
        NumericField('$.runtime', as_name='runtime'),
        TagField('$.day_of_week', as_name='day_of_week')
    )

    # index Definition
    definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.JSON)

    # create Index
    client.ft(INDEX_NAME).create_index(fields=schema, definition=definition)


## Utility function to format the returned documents in tabuler format
def format_result(docs):
  df2 = pd.DataFrame(columns=['Title','Budget','Release Date',
                              'Revenue','Votes','Runtime','Day of Week'])
  for d in docs:
    df2.loc[len(df2.index)] = {"Title": d.Title, "Budget": d.budget,
                             "Release Date": d.release_date, "Revenue": d.revenue,
                             "Votes": d.vote_count, "Runtime": d.runtime, "Day of Week": d.day_of_week}
  return df2

In [ ]:
######################################################## Example 1 ###################################
#@title [Example 2] Get all movies having a maximum budget of $ 1M that released on Friday
##
## Query --> FT.SEARCH idx_movie_2 '@budget:[0 1000000] @day_of_week: {Friday}' RETURN 7 'Title' 'budget' 'release_date' 'revenue' 'vote_count'  'runtime' 'day_of_week'
##
qry = Query('@budget:[1000000 10000000] @day_of_week: {Friday}').return_fields('Title', 'budget', 'release_date', 'revenue', 'vote_count', 'runtime', 'day_of_week')

docs = client.ft(INDEX_NAME).search(qry).docs
format_result(docs)


In [ ]:
############################## Example 2 #############################################################
#@title [Example 3] Get the maximum number of votes which a movie got that released on Friday
##
## Query --> FT.aggregate idx_movie_2 '@day_of_week: {Friday}' groupby 0 reduce max 1 @vote_count as maximum_votes
##
from redis.commands.search.aggregation import AggregateRequest
from redis.commands.search.reducers import max

req = AggregateRequest("@day_of_week: {Friday}").group_by([], max('vote_count').alias("maximum_votes"))
client.ft(INDEX_NAME).aggregate(req).rows



In [ ]:
############################## Example 3 ##############################################################
#@title [Example 4] If you want to find the most popular movie that released on Friday (based on the no of vote it received), use following query:
##
## Query --> FT.SEARCH idx_movie_2 '@day_of_week: {Friday}' sortby 'vote_count' desc RETURN 7 'Title' limit 0 1
##
qry = Query('@day_of_week: {Friday}').sort_by(asc=False, field='vote_count').return_fields('Title').paging(0, 1)
client.ft(INDEX_NAME).search(qry).docs
